In [1]:
import pandas as pd
import os
import spacy
import re

In [2]:
os.chdir('..')

In [3]:
EnTech_df = pd.read_csv("E:\\Zemoso Personal\\NLP_Final\\CleanTech\\NextEra.csv")

In [4]:
EnTech_df['Articles']=EnTech_df['Articles'].apply(str)

In [5]:
# Remove punctuation
EnTech_df['article_text_processed'] = \
EnTech_df['Articles'].map(lambda x: re.sub('[,\.!?]', '', x))

In [7]:
# Convert the titles to lowercase
EnTech_df['article_text_processed'] = \
EnTech_df['article_text_processed'].map(lambda x: x.lower())

In [8]:
EnTech_df['article_text_processed'].head()

0              e n v i r o n m e n t a l   s o c i a l
1              a n d  g o v e r n a n c e  r e p o r t
2    e n v i r o n m e n t a l   s o c i a l  a n d...
3                                        c on t en t s
4    cover photo: the wheatridge renewable energy f...
Name: article_text_processed, dtype: object

In [9]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\punee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','said','world','using','ask_hn','show_hn','new','first','high','could','development','company','patient','study','development','diseases','include','disease','cell','development','medicine','year','research','patient','include','disease','study','forward look', 'company','statement','treatment','cell','medicine','development','other','datum','vertex','research','therapy','clinical','mutation','potential','people','year','phase','age'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [11]:
data = EnTech_df.article_text_processed.values.tolist()
data_words = list(sent_to_words(data))

In [12]:
# remove stop words
data_words = remove_stopwords(data_words)

In [13]:
print(data_words[:1][0][:30])

[]


In [14]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

C:\Users\punee\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


n v r o n m n t l s c i l


In [15]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

[]


In [16]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=25)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams)

[[], [], [], [], ['wheatridge', 'renewable', 'energy', 'facility', 'go', 'online'], ['combine', 'wind', 'solar', 'battery', 'energy'], ['storage', 'technology', 'location', 'wind', 'component', 'encompass'], ['megawatt', 'wind', 'capacity', 'produce', 'approximately', 'wind_turbine'], ['leave', 'catch', 'quick', 'glimpse', 'employee', 'appear'], ['solar_panel', 'equipment', 'check', 'blythe', 'mccoy', 'energy'], [], [], [], ['content'], ['vision_strategy', 'ision'], ['blueprint_reach_real', 'rom', 'ce'], ['report'], ['stakeholder'], ['confront_climate', 'change'], ['honor', 'professional', 'life', 'lead', 'good', 'team', 'industry'], ['team', 'clear', 'vision', 'help', 'lead', 'decarbonization', 'economy', 'vision'], ['bold', 'even', 'unprecedented', 'consistent', 'decade', 'long', 'record', 'performance'], ['portfolio', 'stakeholder', 'include', 'accomplishment', 'goal', 'relate', 'environmental'], ['social', 'governance', 'share', 'report'], ['build_lead_clean_energy', 'provider'], [

In [16]:
pip install --upgrade gensim

In [17]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel
from gensim import corpora

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)
low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

[]


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

C:\Python\lib\site-packages\gensim\models\ldamodel.py:846: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [19]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.06363138), (1, 0.091126606), (2, 0.09129958), (3, 0.058276303), (4, 0.12281322), (5, 0.14445771), (6, 0.079572), (7, 0.13410304), (8, 0.109997585), (9, 0.104722574)]
[(5, 0.14445771), (7, 0.13410304), (4, 0.12281322), (8, 0.109997585), (9, 0.104722574), (2, 0.09129958), (1, 0.091126606), (6, 0.079572), (0, 0.06363138), (3, 0.058276303)]


In [20]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Python\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.349961 -0.217798       1        1  13.618273
7     -0.083957  0.359213       2        1  13.448583
4     -0.325577  0.222551       3        1  12.216582
8      0.227538 -0.313298       4        1  11.344495
9      0.340777 -0.071216       5        1  10.307270
2      0.293247  0.139251       6        1   9.075920
1      0.159585  0.317691       7        1   9.006701
0     -0.061500 -0.362133       8        1   7.974439
6     -0.218537  0.000725       9        1   7.409213
3      0.018385 -0.074985      10        1   5.598525, topic_info=           Term        Freq       Total Category  logprob  loglift
0        energy  426.000000  426.000000  Default  30.0000  30.0000
9          wind  230.000000  230.000000  Default  29.0000  29.0000
310         use  248.000000  248.000000  Default  28.0000  28.0000
137       power  182.000000  182.000000  Default  27.0000  27.0000
365         say  125.000000  125.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
7       combine    9.778394   10.880736  Topic10  -4.9351   2.7758
416   eliminate    9.067870   10.170293  Topic10  -5.0106   2.7679
1851   directly    9.038152   10.140466  Topic10  -5.0139   2.7676
2590  calculate    9.851134   11.063759  Topic10  -4.9277   2.7666
123       begin    8.880534    9.982763  Topic10  -5.0315   2.7657

[333 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
388       6  0.953729       ability
1089      9  0.981822          able
1109      9  0.949113        accord
141       6  0.974230       achieve
208       4  0.950743    additional
...     ...       ...           ...
9         4  0.996775          wind
19        3  0.976580  wind_turbine
80        9  0.981750          work
129       2  0.953031         world
81        2  0.986896          year

[303 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 5, 9, 10, 3, 2, 1, 7, 4])